In [1]:
## Getting to Main directory
import os
os.chdir("../")

In [2]:
# loading secret key
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import pandas as pd

In [4]:
import streamlit as st
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate


e:\projects\AI research assistant\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
load_dotenv()
gemini_api_key=os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

In [69]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import CharacterTextSplitter


In [7]:
csv_file="Data/app_details.csv"

In [8]:
loader = CSVLoader(file_path="Data/app_details.csv")
data = loader.load()

In [9]:
df = pd.read_csv(csv_file)

In [10]:
df.head()

,appId_or_package_name,title,description,summary,installs,score,ratings,developer,categories,icon,headerImage,contentRating,contentRatingDescription,released,lastUpdatedOn,version
0,com.supercell.clashofclans,Clash of Clans,Join millions of players worldwide as you buil...,"Epic combat strategy game. Build your village,...","500,000,000+",4.500125,61148515.0,Supercell,"[{'name': 'Strategy', 'id': 'GAME_STRATEGY'}, ...",https://play-lh.googleusercontent.com/LByrur1m...,https://play-lh.googleusercontent.com/D6u3M1r7...,Rated for 7+,Mild Violence,"Sep 30, 2013","May 3, 2024",16.253.25
1,com.supercell.clashroyale,Clash Royale,Enter the Arena! Build your Battle Deck and ou...,"Clash Royale is a real-time, head-to-head batt...","500,000,000+",4.178366,37525676.0,Supercell,"[{'name': 'Strategy', 'id': 'GAME_STRATEGY'}, ...",https://play-lh.googleusercontent.com/z950eFx-...,https://play-lh.googleusercontent.com/s9qs3woa...,Rated for 7+,Mild Violence,"Mar 1, 2016","Apr 17, 2024",60256021
2,com.dts.freefiremax,Free Fire MAX,[Mechadrake Incoming]\r\nThe fierce Mechadrake...,MAX Battle Royale Experience,"100,000,000+",4.300161,22448961.0,Garena International I,"[{'name': 'Action', 'id': 'GAME_ACTION'}, {'na...",https://play-lh.googleusercontent.com/T8RuSQDF...,https://play-lh.googleusercontent.com/PTF7b77h...,Rated for 12+,Moderate Violence,"Sep 18, 2021","Apr 7, 2024",2.104.1
3,com.topgamesinc.evony,Evony: The King's Return,Shoot and engage in battles！\r\nBuild your cit...,"Move and Shoot! Build Your Empire, Explore Dif...","100,000,000+",3.536870,705629.0,TG Inc.,"[{'name': 'Strategy', 'id': 'GAME_STRATEGY'}, ...",https://play-lh.googleusercontent.com/_AnSpAUp...,https://play-lh.googleusercontent.com/0WahHxuN...,Rated for 7+,Mild Violence,"Aug 10, 2016","May 15, 2024",4.72.2
4,com.vizorapps.klondike,Klondike Adventures,Do you like travelling to odd places? 🗺 Fancy ...,Farming games: Harvest crops and Grow your own...,"50,000,000+",4.055430,1232980.0,VIZOR APPS LTD.,"[{'name': 'Simulation', 'id': 'GAME_SIMULATION...",https://play-lh.googleusercontent.com/jW61KFsl...,https://play-lh.googleusercontent.com/vyiC4Rx0...,Rated for 3+,NaN,"Apr 3, 2018","May 10, 2024",2.121.1


In [11]:
from datetime import datetime
from typing import List
import csv
from langchain.docstore.document import Document
from langchain.document_loaders.base import BaseLoader
from typing import Optional, Dict


In [12]:
import pandas as pd
import langchain  # Assuming Langchain is installed

class CsvDocumentLoader:
    """A document loader class that reads CSV data into Langchain documents using a DataFrame."""

    def __init__(self, file_path, content_column="description", metadata_columns=None):
        """
        Initializes the document loader.

        Args:
            file_path (str): Path to the CSV file.
            content_column (str, optional): Name of the column containing document content. Defaults to "description".
            metadata_columns (List[str], optional): List of column names for additional metadata. Defaults to None, which includes all columns except the content column.
        """

        self.file_path = file_path
        self.content_column = content_column
        self.metadata_columns = metadata_columns

    def load(self) -> List[Document]:
        """
        Loads CSV data as a list of Langchain documents.

        Returns:
            List[Document]: List of Langchain documents created from the CSV data.
        """

        try:
            # Read CSV data into a DataFrame
            df = pd.read_csv(self.file_path)

            # Extract content and metadata
            content = df[self.content_column].tolist()  # List of content strings
            if self.metadata_columns is None:
                # Include all columns except content for metadata
                metadata_columns = list(df.columns)
                metadata_columns.remove(self.content_column)
                metadata = df[metadata_columns].to_dict(orient="records")  # List of metadata dictionaries
            else:
                # Use specified columns for metadata
                metadata = df[self.metadata_columns].to_dict(orient="records")

            # Create Langchain documents
            docs = []
            for i, (cont, meta) in enumerate(zip(content, metadata)):
                doc = Document(page_content=cont.strip(), metadata=dict(meta, row=i))  # Add row number to metadata
                docs.append(doc)

            return docs

        except FileNotFoundError as e:
            print(f"Error: File not found: {self.file_path}")
            return []
        except Exception as e:
            print(f"Unexpected error: {str(e)}")
            return []

# Usage example
# loader = CsvDocumentLoader("your_csv_file.csv", content_column="description", metadata_columns=["title", "developer"])
# documents = loader.load()

# # Process documents using Langchain
# for doc in documents:
#     print(doc.page_content)
#     print(doc.metadata)


In [13]:
metadata_columns_list = [
    "appId_or_package_name", "title", "summary", "installs", "score",
    "ratings", "developer", "categories", "icon", "headerImage",
    "contentRating", "contentRatingDescription", "released", "lastUpdatedOn",
    "version"
]


In [14]:
loader = CsvDocumentLoader(file_path=csv_file, content_column="description",  metadata_columns=metadata_columns_list)
docs = loader.load()
docs[0]

Document(page_content="Join millions of players worldwide as you build your village, raise a clan, and compete in epic Clan Wars!\r\n\r\nMustachioed Barbarians, fire wielding Wizards, and other unique troops are waiting for you! Enter the world of Clash!\r\n\r\nClassic Features:\r\n● Join a Clan of fellow players or start your own and invite friends.\r\n● Fight in Clan Wars as a team against millions of active players across the globe.\r\n● Test your skills in the competitive Clan War Leagues and prove you’re the best.\r\n● Forge alliances, work together with your Clan in Clan Games to earn valuable Magic items.\r\n● Plan your unique battle strategy with countless combinations of Spells, Troops, and Heroes!\r\n● Compete with the best players from around the world and rise to the top of the Leaderboard in Legend League.\r\n● Collect resources and steal loot from other players to upgrade your own Village and turn it into a stronghold. \r\n● Defend against enemy attacks with a multitude o

In [15]:
len(docs)


58

In [70]:
# Transform data
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
texts = text_splitter.split_documents(data)

In [71]:

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=10)
# text_chunks = text_splitter.split_documents(docs)

In [72]:
text_chunks[:5]

[Document(page_content='Join millions of players worldwide as you build your village, raise a clan, and compete in epic Clan Wars! Mustachioed Barbarians, fire wielding Wizards, and other unique troops are waiting for you! Enter the world of Clash! Classic Features: ● Join a Clan of fellow players or start your own and invite friends. ● Fight in Clan Wars as a team against millions of active players across the globe. ● Test your skills in the competitive Clan War Leagues and prove you’re the best. ● Forge alliances, work together with your Clan in Clan Games to earn valuable Magic items. ● Plan your unique battle strategy with countless combinations of Spells, Troops, and Heroes! ● Compete with the best players from around the world and rise to the top of the Leaderboard in Legend League. ● Collect resources and steal loot from other players to upgrade your own Village and turn it into a stronghold.', metadata={'appId_or_package_name': 'com.supercell.clashofclans', 'title': 'Clash of C

In [73]:
len(text_chunks)


167

In [109]:

from langchain_google_genai import GoogleGenerativeAIEmbeddings

from langchain.prompts.chat import ChatPromptTemplate

In [110]:
model = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro-latest", api_key=gemini_api_key, temperature=0.3,  convert_system_message_to_human=True
)

genai.configure(api_key=gemini_api_key)


In [74]:
# Clean up our Documents' content
import re

def clean_up_text(content: str) -> str:
    """
    Remove unwanted characters and patterns in text input.

    :param content: Text input.
    
    :return: Cleaned version of original text input.
    """

    # Fix hyphenated words broken by newline
    content = re.sub(r'(\w+)-\n(\w+)', r'\1\2', content)

    # Remove specific unwanted patterns and characters
    unwanted_patterns = [
        "\\n", "  —", "——————————", "—————————", "—————",
        r'\\u[\dA-Fa-f]{4}', r'\uf075', r'\uf0b7'
    ]
    for pattern in unwanted_patterns:
        content = re.sub(pattern, "", content)

    # Fix improperly spaced hyphenated words and normalize whitespace
    content = re.sub(r'(\w)\s*-\s*(\w)', r'\1-\2', content)
    content = re.sub(r'\s+', ' ', content)

    return content

# Call function
cleaned_docs = []
for d in text_chunks: 
    cleaned_text = clean_up_text(d.page_content)
    d.page_content = cleaned_text
    cleaned_docs.append(d)


In [75]:
text_chunks[:5]


[Document(page_content='Join millions of players worldwide as you build your village, raise a clan, and compete in epic Clan Wars! Mustachioed Barbarians, fire wielding Wizards, and other unique troops are waiting for you! Enter the world of Clash! Classic Features: ● Join a Clan of fellow players or start your own and invite friends. ● Fight in Clan Wars as a team against millions of active players across the globe. ● Test your skills in the competitive Clan War Leagues and prove you’re the best. ● Forge alliances, work together with your Clan in Clan Games to earn valuable Magic items. ● Plan your unique battle strategy with countless combinations of Spells, Troops, and Heroes! ● Compete with the best players from around the world and rise to the top of the Leaderboard in Legend League. ● Collect resources and steal loot from other players to upgrade your own Village and turn it into a stronghold.', metadata={'appId_or_package_name': 'com.supercell.clashofclans', 'title': 'Clash of C

In [76]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings


In [77]:
genai.configure(api_key=gemini_api_key)
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
# Setting Tempreture to 0.3 for getting low risk results

model = ChatGoogleGenerativeAI(model="gemini-1.5-pro-latest",api_key=gemini_api_key,temperature=0.3)

In [115]:
# Fill Vector DB
docsearch = FAISS.from_documents(texts, embeddings)
retriever=docsearch.as_retriever()

In [116]:
ans = docsearch.similarity_search("Which games may you recommend for adventure lover?")

ans[0]


Document(page_content="appId_or_package_name: com.vizorapps.klondike\ntitle: Klondike Adventures\ndescription: Do you like travelling to odd places? 🗺 Fancy renovating abandoned spots? ⚒️ Or maybe you just want to take a break and play a relaxing mini-game? 👾\r\n\r\nKlondike has it all! Join in for so much fun:\r\n\r\n🍿 <b>FOLLOW THE STORY</b> of Paul and Kate going through quite a life! This is going to feel like your favorite TV show, we promise!\r\n\r\n💫 <b>EXPLORE</b> numerous locations set in various landscapes and having their own distinct content. This game has it all! 🤩\r\n\r\n🏘 <b>BUILD</b> a flourishing city with factories & farms, mine for resources and craft everything you need for travels, quests, new buildings and orders from the locals dropping by your town.\r\n\r\n🐏 <b>FARM</b> simulation in the wild! Raise animals & harvest crops, make food for yourself and to trade.\r\n\r\n🏆 <b>COMPETE</b> with neighboring farm owners & take as many craft and farm orders as you can du

In [117]:
docs = retriever.get_relevant_documents("Which games may you recommend for adventure lover?")

docs[0]

Document(page_content="appId_or_package_name: com.vizorapps.klondike\ntitle: Klondike Adventures\ndescription: Do you like travelling to odd places? 🗺 Fancy renovating abandoned spots? ⚒️ Or maybe you just want to take a break and play a relaxing mini-game? 👾\r\n\r\nKlondike has it all! Join in for so much fun:\r\n\r\n🍿 <b>FOLLOW THE STORY</b> of Paul and Kate going through quite a life! This is going to feel like your favorite TV show, we promise!\r\n\r\n💫 <b>EXPLORE</b> numerous locations set in various landscapes and having their own distinct content. This game has it all! 🤩\r\n\r\n🏘 <b>BUILD</b> a flourishing city with factories & farms, mine for resources and craft everything you need for travels, quests, new buildings and orders from the locals dropping by your town.\r\n\r\n🐏 <b>FARM</b> simulation in the wild! Raise animals & harvest crops, make food for yourself and to trade.\r\n\r\n🏆 <b>COMPETE</b> with neighboring farm owners & take as many craft and farm orders as you can du

In [118]:
qa = RetrievalQA.from_chain_type(llm=model, chain_type="stuff", retriever=retriever)
qa.run({"query": "Which games may you recommend for adventure lover?"} )


'Based on the provided descriptions, here are the games that might suit an adventure lover:\n\n* **Klondike Adventures:** This game offers a blend of adventure, exploration, and city-building. You follow a story, explore new locations, solve mysteries, and build up towns. \n* **Fortias Saga: Action Adventure:** This game features a classic RPG adventure where you battle monsters, explore diverse worlds, collect resources, and recruit heroes to fight against dark forces. \n* **Puzzle Legends:** While this game focuses on puzzle-solving, it also features a storyline where you lead a group of heroes against an evil army, explore dungeons, and face unique challenges. \n\nThe choice depends on what kind of adventure the person prefers: story-driven exploration, strategic combat, or puzzle-based challenges with a narrative. \n'

In [121]:
qa.run({"query": "Which games may you recommend for adventure lover give me 10 recommendation?, give me image url too"} )


'Here are 10 adventure games with their icon image URLs:\n\n1. **Klondike Adventures**\n   - Description: Embark on a journey with Paul and Kate, exploring diverse landscapes, building a flourishing city, and farming in the wild.\n   - Icon URL: https://play-lh.googleusercontent.com/jW61KFsluyqSfGV_GhBNWdNJfmNfF8vApum1WiWw2hNR7cirY4hIaGd6fQ2zUGomRXw\n\n2. **Fortias Saga: Action Adventure**\n   - Description: Engage in a war against dark forces, collecting resources, recruiting heroes, and battling monsters across various worlds.\n   - Icon URL: https://play-lh.googleusercontent.com/-T2g7Lcbim190MJ_g7qnZmTn7zjpdrkryPeHSanFq81Je5MJUeG_0ah3KiqdesC_LeU\n\n3. **Clash of Clans**\n   - Description: Build your village, raise a clan, and compete in epic Clan Wars in this popular strategy game.\n   - Icon URL: https://play-lh.googleusercontent.com/LByrur1mTmPeNr0ljI-uAUcct1rzmTve5Esau1SwoAzjBXQUby6uHIfHbF9TAT51mgHm\n\n4. **Mythic Samkok**\n   - Description: Experience the Three Kingdoms era with

In [81]:
# vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)
# vector_store.save_local("faiss_index")

In [95]:
prompt_template = """
Answer the question as detailed as possible from the provided context and the metadata, make sure to provide all the details
provided context just say, "answer is not available in the context", don't provide the wrong answer\n\n
Context:\n {context}?\n
Question: \n{question}\n


Instructions for Game Recommendation:

1. Provide a brief overview of the user's gaming preferences or requirements, if available.
2. Suggest game titles based on the user's preferences, considering factors such as genre, platform, gameplay style, and user ratings.
3. Include a concise description of each recommended game, highlighting its key features, gameplay mechanics, and any unique selling points.
4. If applicable, mention any additional information relevant to the user's query, such as in-game purchases, multiplayer options, or compatibility with specific devices.
5. Aim to offer a diverse selection of game recommendations to cater to different tastes and preferences.
6. Avoid recommending games that may not align with the user's preferences or requirements.


Answer:
"""

# model = ChatGoogleGenerativeAI(model="gemini-pro",
#                             temperature=0.3)

prompt = PromptTemplate(template = prompt_template, input_variables = ["context", "question"])

prompt
# return chain

PromptTemplate(input_variables=['context', 'question'], template='\nAnswer the question as detailed as possible from the provided context and the metadata, make sure to provide all the details\nprovided context just say, "answer is not available in the context", don\'t provide the wrong answer\n\n\nContext:\n {context}?\n\nQuestion: \n{question}\n\n\n\nInstructions for Game Recommendation:\n\n1. Provide a brief overview of the user\'s gaming preferences or requirements, if available.\n2. Suggest game titles based on the user\'s preferences, considering factors such as genre, platform, gameplay style, and user ratings.\n3. Include a concise description of each recommended game, highlighting its key features, gameplay mechanics, and any unique selling points.\n4. If applicable, mention any additional information relevant to the user\'s query, such as in-game purchases, multiplayer options, or compatibility with specific devices.\n5. Aim to offer a diverse selection of game recommendation

In [96]:
from langchain.prompts.chat import ChatPromptTemplate
qa_prompt = ChatPromptTemplate.from_messages([("human", prompt_template)])
qa_prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='\nAnswer the question as detailed as possible from the provided context and the metadata, make sure to provide all the details\nprovided context just say, "answer is not available in the context", don\'t provide the wrong answer\n\n\nContext:\n {context}?\n\nQuestion: \n{question}\n\n\n\nInstructions for Game Recommendation:\n\n1. Provide a brief overview of the user\'s gaming preferences or requirements, if available.\n2. Suggest game titles based on the user\'s preferences, considering factors such as genre, platform, gameplay style, and user ratings.\n3. Include a concise description of each recommended game, highlighting its key features, gameplay mechanics, and any unique selling points.\n4. If applicable, mention any additional information relevant to the user\'s query, such as in-game purchases, multiplayer opti

In [97]:
chain = load_qa_chain(llm=model, chain_type="stuff", verbose=True, prompt=qa_prompt)


In [98]:
user_question = "Which games may you recommend for adventure lover? "


In [99]:
new_db = FAISS.load_local("faiss_index", embeddings,allow_dangerous_deserialization=True)
docs = new_db.similarity_search(user_question)

In [100]:
docs


[Document(page_content='Twitter : https://twitter.com/BrownDust2_EN', metadata={'appId_or_package_name': 'com.neowizgames.game.browndust2', 'title': 'BrownDust2 - Adventure RPG', 'summary': 'Enjoyable JRPG Masterpiece with High End 2D Graphics', 'installs': '1,000,000+', 'score': 4.49, 'ratings': 63295.0, 'developer': 'NEOWIZ', 'categories': "[{'name': 'Role Playing', 'id': 'GAME_ROLE_PLAYING'}, {'name': 'Turn-based RPG', 'id': None}, {'name': 'Multiplayer', 'id': None}, {'name': 'Competitive multiplayer', 'id': None}, {'name': 'Single player', 'id': None}, {'name': 'Stylized', 'id': None}, {'name': 'Anime', 'id': None}]", 'icon': 'https://play-lh.googleusercontent.com/vl_iLdKz4F84jKFTa7acfDYe-Uu8zqx4GWKFnSJDViCX_gIIX5RYZYrpnfgouWtvtCk', 'headerImage': 'https://play-lh.googleusercontent.com/dfR5al1SHyNPIQM6wb7G7X53U9TseR6DZgucT2Bb8mrycza1fN_q9PpCdURhIQjRUgE', 'contentRating': 'Rated for 12+', 'contentRatingDescription': 'Sexual Innuendo', 'released': 'Jun 15, 2023', 'lastUpdatedOn': 'M

In [101]:
response = chain(
        {"input_documents":docs, "question": user_question}
        , return_only_outputs=True)



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Human: 
Answer the question as detailed as possible from the provided context and the metadata, make sure to provide all the details
provided context just say, "answer is not available in the context", don't provide the wrong answer


Context:
 Twitter : https://twitter.com/BrownDust2_EN

Terms of Use: http://www.gameloft.com/en/conditions-of-use Privacy Policy: http://www.gameloft.com/en/privacy-notice End-User License Agreement: http://www.gameloft.com/en/eula

To have an excellent of music game experience, it is recommended to use headphones. More surprises are waiting for you! Download now and enjoy a new gaming experience!

Privacy Policy: https://elementalraiders.gamesforaliving.com/privacy-policy/ End-User License Agreement:https://elementalraiders.gamesforaliving.com/tou/?

Question: 
Which games may you recommend for adventure lover? 



Instructions for Game Recommendatio

In [102]:
print(response["output_text"])


Answer is not available in the context. 



In [45]:

from langchain_google_genai import GoogleGenerativeAIEmbeddings

from langchain.prompts.chat import ChatPromptTemplate

In [52]:
model = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro-latest", api_key=gemini_api_key, temperature=0.3,  convert_system_message_to_human=True
)

genai.configure(api_key=gemini_api_key)


In [53]:
docsearch = Chroma.from_documents(text_chunks,embeddings)

In [54]:
from langchain.chains import RetrievalQA

In [103]:
qa = RetrievalQA.from_chain_type(llm=model,chain_type="stuff",retriever=docsearch.as_retriever())

In [104]:
qa.run("recommend me 2 adventure games")

ValueError: SystemMessages are not yet supported!

To automatically convert the leading SystemMessage to a HumanMessage,
set  `convert_system_message_to_human` to True. Example:

llm = ChatGoogleGenerativeAI(model="gemini-pro", convert_system_message_to_human=True)


In [58]:
from langchain.text_splitter import CharacterTextSplitter


In [59]:
# Transform data
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

In [61]:
# Fill Vector DB
docsearch = FAISS.from_documents(texts, embeddings)
retriever=docsearch.as_retriever()

In [64]:
ans = docsearch.similarity_search("Which games may you recommend for adventure lover?")

ans


[Document(page_content="appId_or_package_name: com.vizorapps.klondike\ntitle: Klondike Adventures\ndescription: Do you like travelling to odd places? 🗺 Fancy renovating abandoned spots? ⚒️ Or maybe you just want to take a break and play a relaxing mini-game? 👾\r\n\r\nKlondike has it all! Join in for so much fun:\r\n\r\n🍿 <b>FOLLOW THE STORY</b> of Paul and Kate going through quite a life! This is going to feel like your favorite TV show, we promise!\r\n\r\n💫 <b>EXPLORE</b> numerous locations set in various landscapes and having their own distinct content. This game has it all! 🤩\r\n\r\n🏘 <b>BUILD</b> a flourishing city with factories & farms, mine for resources and craft everything you need for travels, quests, new buildings and orders from the locals dropping by your town.\r\n\r\n🐏 <b>FARM</b> simulation in the wild! Raise animals & harvest crops, make food for yourself and to trade.\r\n\r\n🏆 <b>COMPETE</b> with neighboring farm owners & take as many craft and farm orders as you can d

In [66]:
docs = retriever.get_relevant_documents("Which games may you recommend for adventure lover?")

docs

[Document(page_content="appId_or_package_name: com.vizorapps.klondike\ntitle: Klondike Adventures\ndescription: Do you like travelling to odd places? 🗺 Fancy renovating abandoned spots? ⚒️ Or maybe you just want to take a break and play a relaxing mini-game? 👾\r\n\r\nKlondike has it all! Join in for so much fun:\r\n\r\n🍿 <b>FOLLOW THE STORY</b> of Paul and Kate going through quite a life! This is going to feel like your favorite TV show, we promise!\r\n\r\n💫 <b>EXPLORE</b> numerous locations set in various landscapes and having their own distinct content. This game has it all! 🤩\r\n\r\n🏘 <b>BUILD</b> a flourishing city with factories & farms, mine for resources and craft everything you need for travels, quests, new buildings and orders from the locals dropping by your town.\r\n\r\n🐏 <b>FARM</b> simulation in the wild! Raise animals & harvest crops, make food for yourself and to trade.\r\n\r\n🏆 <b>COMPETE</b> with neighboring farm owners & take as many craft and farm orders as you can d

In [68]:
qa = RetrievalQA.from_chain_type(llm=model, chain_type="stuff", retriever=retriever)
qa.run({"query": "Which games may you recommend for adventure lover?"} )


'Based on the provided descriptions, here are the games that might appeal to an adventure lover:\n\n* **Klondike Adventures:** This game offers a mix of adventure, exploration, and building. You follow the story of Paul and Kate as they travel to different locations, solve mysteries, and build up ancient ruins. \n* **Fortias Saga: Action Adventure:** This RPG features a classic adventure storyline where you battle monsters, explore various worlds, and recruit heroes to fight against dark forces. \n'